In [5]:
'''from pylab import *
import h5py, os, sys, traceback
from glob import glob
from IPython.core.debugger import Tracer; set_trace = Tracer()

from polprocess import file_newer, load_cal, process
from sunpos import sunpos_az as sunpos, arcdist
from simulate import oceanstokes, oceanaop
from polarization import *
from plot_utils import *
from stats_utils import *
from geomag.emm import EMMMesh

from datetime import datetime, timedelta, timezone
import scipy.stats as stats
from scipy.optimize import minimize
from scipy.spatial import cKDTree as KDTree
from geographiclib.geodesic import Geodesic
from mpl_toolkits.basemap import Basemap
from pint import UnitRegistry
import yaml
units = UnitRegistry()
WGS84 = Geodesic.WGS84

from scipy.optimize import minimize
import math
from simulate import oceanstokes, oceanaop'''

/Users/adityamantha/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`
  after removing the cwd from sys.path.


SyntaxError: invalid syntax (polprocess.py, line 182)

In [1]:
from scipy.optimize import minimize
import math
from simulate import oceanstokes, oceanaop

pi = math.pi
def angle_diff(a1,a2,period=2*pi):
    """(a1 - a2 + d) % (2*d) - d; d = period/2"""
    d = period/2
    return ((a1 - a2 + d) % (period)) - d
def sun_pos_error_l1(sun_head,sun_zen,cam_aop,cam_head,cam_pitch,m2):
    sim_aop = oceanaop(sun_head,sun_zen,cam_head,cam_pitch,m2)
    d = angle_diff(cam_aop,sim_aop,pi)
    return (abs(d))

sun_pos_error = sun_pos_error_l1
def fit_sun_head_to_aop(aop,head,pitch,ridx=1.33,verbose=False):
    #fit the sun heading to the data by assuming the sun zenith angle is pi/4:
    fit = minimize(sun_pos_error,x0=0,args=(pi/4,aop,head,pitch,ridx),bounds=[(-2*pi,2*pi)],options={'gtol':1e-6})
    return fit.x
def fit_sun_to_aop(aop,head,pitch,sun_head_guess=None,ridx=1.33,verbose=False,vl=0):
    """fit_sun_to_aop, find sun position corresponding to aops at different headings & pitches, no time passing
    aop: array of AoPs, radians
    head: array of headings, radians, same shape as aop
    pitch: array of pitchs, radians, same shape as head
    ridx: refractive index of water
    """
    v = verbose
    #_p(v,vl,'Fitting sun ',end='')
    if sun_head_guess is None:
        #_p(v,0,'heading to AoP... ',end='',flush=True)
        #fit just the heading first to get a good initial point:
        sun_head_guess = fit_sun_head_to_aop(aop,head,pitch,ridx)
    #_p(v,0,'heading and zenith to AoP... ',end='',flush=True)
    #now do both heading & zenith
    minfun = lambda x,*args: sun_pos_error(x[0],x[1],*args)
    fit = minimize(minfun,x0=(sun_head_guess,pi/4),args=(aop,head,pitch,ridx),bounds=[(-2*pi,2*pi),(.01,pi/2)],options={'gtol':1e-6})
    sun_hz = fit.x
    #_p(v,0,'DONE',flush=True)
    return sun_hz

In [2]:
fit_sun_to_aop(1.645,0,1.57079636)

array([-1.49659272,  0.78539816])